In [2]:
import pandas as pd
import numpy as np
import sklearn

import warnings
#warnings.filterwarnings("ignore")

from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score

from copy import deepcopy

import xgboost as xgb

N_FOLDS = 3

## Загружаем данные, делим их на фолды

In [4]:
df_train  = pd.read_csv('./ngwl-predict-customer-churn/train/train.csv')
#df_train
df_train['order_completed_at'] = pd.to_datetime(df_train['order_completed_at'])

df_test = pd.read_csv("./ngwl-predict-customer-churn/sample_submission.csv", sep=';')
df_test

,Id,Predicted
0,19843,NaN
1,471287,NaN
2,342522,NaN
3,457799,NaN
4,233778,NaN
...,...,...
214604,16841,NaN
214605,69348,NaN
214606,594974,NaN
214607,320024,NaN


In [5]:
train_data_def = pd.read_csv('./ngwl-predict-customer-churn/train/train.csv', sep=',')
#print(train_data_def)

train_start = '2020-06'
test_start = '2020-07'

train_data = train_data_def[train_data_def['order_completed_at']==train_start]
print(train_data['phone_id'].nunique())
print(train_data['target'].value_counts())

test_data = train_data_def[train_data_def['order_completed_at']==test_start]
print(test_data['phone_id'].nunique())
print(test_data['target'].value_counts())

train_data.loc[:, 'fold_num'] = [np.random.randint(1,N_FOLDS+1) for i in range(len(train_data))]
train_data['fold_num'].value_counts()
train_data['month']=6

test_data.loc[:, 'fold_num'] = [np.random.randint(1,N_FOLDS+1) for i in range(len(test_data))]
test_data['fold_num'].value_counts()
test_data['month']=7


train_folds = []
test_folds = []

for i in range(1, N_FOLDS+1):
    local_data_train = train_data.loc[train_data['fold_num']==i]
    local_data_test = test_data.loc[test_data['fold_num']==i]

    del local_data_train['fold_num']
    del local_data_test['fold_num']
    
    del local_data_train['order_completed_at']
    del local_data_test['order_completed_at']
    
    train_folds.append(local_data_train.reset_index(drop=True))
    test_folds.append(local_data_test.reset_index(drop=True))

254285
0.0    127797
1.0    126488
Name: target, dtype: int64
284449
0.0    151807
1.0    132642
Name: target, dtype: int64


/home/dana_zl/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/dana_zl/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/home/dana_zl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [6]:
#генерируем признак rate_mean c оценкой 3 как нейтральное значение
# shipments
shipments = pd.DataFrame({})
for i in ['shipments2020-01-01', 'shipments2020-03-01', 'shipments2020-04-30', 'shipments2020-06-29']:
    tmp = pd.read_csv(f'ngwl-predict-customer-churn/shipments/{i}.csv')
    shipments = pd.concat([tmp, shipments])
    
adress = pd.read_csv('ngwl-predict-customer-churn/misc/addresses.csv')
# add users phone
shipments = shipments.merge(adress, how='left', left_on='ship_address_id', right_on='id')

# month
shipments['dt_'] = pd.to_datetime(shipments['order_completed_at'], format='%Y-%m-%d %H:%M:%S')
shipments = shipments[shipments['dt_']>='2020-01-01']
shipments['month'] = pd.to_datetime(shipments['order_completed_at'], format='%Y-%m-%d %H:%M:%S').dt.month
shipments = shipments.drop_duplicates()
#shipments_first = shipments.groupby(by='phone_id', as_index=False)['dt_'].min()
#shipments_first['first_month'] = shipments_first['dt_'].dt.month
shipments['rate'] = shipments['rate'].replace({0:3})
f_shipments = shipments.groupby(by=['phone_id', 'month'], as_index=False).agg({
    'rate': ['mean'],

})
f_shipments.columns = [x[0]+'_'+x[1] + "_new" if 
                       not ((x[0] =='phone_id') or (x[0] =='month'))
                       else x[0]
                       for x in f_shipments.columns]
f_shipments


,phone_id,month,rate_mean_new
0,2,1,3.0
1,2,2,3.0
2,2,3,3.0
3,2,5,3.0
4,3,3,5.0
...,...,...,...
1155779,719367,8,3.0
1155780,719369,8,5.0
1155781,719370,8,5.0
1155782,719373,8,3.0


In [7]:
#загружаем заранее собранные признаки для клиентов и мерджим с новой фичей
client_features = pd.read_pickle("./fea_all_actual_01_38.pkl")
client_features = client_features.merge(f_shipments, on=['month', 'phone_id'])
client_features

,order_id_nunique,total_cost_sum,total_cost_min,total_cost_max,total_cost_mean,total_cost_std,rate_mean,promo_total_sum,promo_total_mean,total_weight_sum,...,Санкт-Петербург,discount_msg_sum,bonus_msg_sum,spam_msg_sum,email_sum,push_sum,sms_sum,action_id_nunique,gap_in_month_before,rate_mean_new
0,1,98.0,98.0,98.0,98.0,0.000000,0.0,0.000000,0.000000,22024,...,0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1,3.0
1,2,366.0,158.0,208.0,183.0,35.355339,0.0,0.000000,0.000000,67946,...,0,0.0,0.0,3.0,3.0,0.0,0.0,3.0,0,3.0
2,1,233.0,233.0,233.0,233.0,0.000000,0.0,0.000000,0.000000,34195,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,3.0
3,1,248.0,248.0,248.0,248.0,0.000000,0.0,0.000000,0.000000,40777,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3.0
4,2,0.0,0.0,0.0,0.0,0.000000,5.0,0.000000,0.000000,31265,...,0,0.0,0.0,3.0,3.0,0.0,0.0,3.0,1,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1155779,1,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,5320,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3.0
1155780,1,0.0,0.0,0.0,0.0,0.000000,5.0,0.000000,0.000000,12270,...,0,1.0,3.0,12.0,6.0,8.0,2.0,16.0,1,5.0
1155781,1,0.0,0.0,0.0,0.0,0.000000,5.0,-163.770004,-163.770004,14039,...,1,0.0,4.0,14.0,9.0,9.0,2.0,18.0,1,5.0
1155782,1,0.0,0.0,0.0,0.0,0.000000,0.0,-193.270004,-193.270004,18470,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3.0


In [8]:
#собираем датасеты из признаков для клиентов за разные месяца - по одному месяцу, по 2 и по 3
# в случае нескольких столбцов признаки за месяца конкатенируются
client_features_6 = client_features[client_features['month'] == 6]
client_features_7 = client_features[client_features['month'] == 7]
client_features_56 = pd.merge(client_features[client_features['month'] == 5],
                              client_features[client_features['month'] == 6],
         how='outer', on='phone_id')

client_features_34 = pd.merge(client_features[client_features['month'] == 3],
                              client_features[client_features['month'] == 4],
                              how='outer', on='phone_id'
) 

client_features_45 = pd.merge(client_features[client_features['month'] == 4],
                              client_features[client_features['month'] == 5],
                              how='outer', on='phone_id'
)   
client_features_67 = pd.merge(
    client_features[client_features['month'] == 6],
    client_features[client_features['month'] == 7],
    how='outer', on='phone_id'
)

client_features_78 = pd.merge(
    client_features[client_features['month'] == 7],
    client_features[client_features['month'] == 8],
    how='outer', on='phone_id'
)  

client_features_345 = pd.merge(
    client_features_34,
    client_features[client_features['month'] == 5],
    how='outer', on='phone_id'
)

client_features_678 = pd.merge(
    client_features_67,
    client_features[client_features['month'] == 8],
    how='outer', on='phone_id'
)
                              
client_features_567 = pd.merge(client_features_56, client_features[client_features['month'] == 7],
         how='outer', on='phone_id')
client_features_456 = pd.merge(client_features_45, client_features[client_features['month'] == 6],
         how='outer', on='phone_id')

client_features_456['delta_order_cnt_12'] = client_features_456['order_id_nunique_y'] - client_features_456['order_id_nunique_x']
client_features_456['delta_order_cnt_23'] = client_features_456['order_id_nunique'] - client_features_456['order_id_nunique_y']
client_features_456['delta_order_state_complete_12'] = client_features_456['order_state_complete_y'] \
- client_features_456['order_state_complete_x']
client_features_456['delta_order_state_complete_23'] = client_features_456['order_state_complete'] \
- client_features_456['order_state_complete_y']
client_features_456['delta_price_sum_12'] = client_features_456['price_sum_y'] \
- client_features_456['price_sum_x']
client_features_456['delta_price_sum_23'] = client_features_456['price_sum'] \
- client_features_456['price_sum_y']

client_features_567['delta_order_cnt_12'] = client_features_567['order_id_nunique_y'] - client_features_567['order_id_nunique_x']
client_features_567['delta_order_cnt_23'] = client_features_567['order_id_nunique'] - client_features_567['order_id_nunique_y']
client_features_567['delta_order_state_complete_12'] = client_features_567['order_state_complete_y'] \
- client_features_567['order_state_complete_x']
client_features_567['delta_order_state_complete_23'] = client_features_567['order_state_complete'] \
- client_features_567['order_state_complete_y']
client_features_567['delta_price_sum_12'] = client_features_567['price_sum_y'] \
- client_features_567['price_sum_x']
client_features_567['delta_price_sum_23'] = client_features_567['price_sum'] \
- client_features_567['price_sum_y']

client_features_678['delta_order_cnt_12'] = client_features_678['order_id_nunique_y'] - client_features_678['order_id_nunique_x']
client_features_678['delta_order_cnt_23'] = client_features_678['order_id_nunique'] - client_features_678['order_id_nunique_y']
client_features_678['delta_order_state_complete_12'] = client_features_678['order_state_complete_y'] \
- client_features_678['order_state_complete_x']
client_features_678['delta_order_state_complete_23'] = client_features_678['order_state_complete'] \
- client_features_678['order_state_complete_y']
client_features_678['delta_price_sum_12'] = client_features_678['price_sum_y'] \
- client_features_678['price_sum_x']
client_features_678['delta_price_sum_23'] = client_features_678['price_sum'] \
- client_features_678['price_sum_y']

client_features_567

,order_id_nunique_x,total_cost_sum_x,total_cost_min_x,total_cost_max_x,total_cost_mean_x,total_cost_std_x,rate_mean_x,promo_total_sum_x,promo_total_mean_x,total_weight_sum_x,...,sms_sum,action_id_nunique,gap_in_month_before,rate_mean_new,delta_order_cnt_12,delta_order_cnt_23,delta_order_state_complete_12,delta_order_state_complete_23,delta_price_sum_12,delta_price_sum_23
0,1.0,248.0,248.0,248.0,248.0,0.000000,0.00,0.0,0.0,40777.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,168.0,168.0,168.0,168.0,0.000000,1.00,-250.0,-250.0,25106.0,...,0.0,25.0,1.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,331.0,163.0,168.0,165.5,3.535534,2.00,0.0,0.0,56605.0,...,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,-2924.440041,NaN
3,4.0,316.0,0.0,158.0,79.0,91.221343,1.25,-600.0,-150.0,57316.0,...,0.0,1.0,0.0,3.0,-2.0,1.0,-2.0,1.0,-9982.900042,6533.700037
4,1.0,158.0,158.0,158.0,158.0,0.000000,5.00,0.0,0.0,3990.0,...,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,1388.239992,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,7.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
404033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,14.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
404034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,22.0,1.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN
404035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,3.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# удаляем неважные признаки (были выделены по feature_importance в прошлых экспериментах)
# и обучаем модель на фолдах
clf = xgb.XGBClassifier(n_estimators=1000, max_depth=6, learning_rate=0.01, random_state=42)

for idx in range(N_FOLDS):
    train_fold = train_folds[idx]
    test_fold = test_folds[idx]
    
    train_features = pd.merge(train_fold, client_features_456, on='phone_id')
    train_features[['rate_mean_new', 'rate_mean_new_x', 'rate_mean_new_y']] = train_features[['rate_mean_new', 'rate_mean_new_x', 'rate_mean_new_y']].fillna(3)
    train_features = train_features.fillna(0)
    print(len(train_fold), len(train_features), train_features.shape)
       
    #print(train_features)
    train_target = deepcopy(train_features['target'])
    del train_features['target']
    del train_features['phone_id']
    del train_features['month_x']
    del train_features['month_y']
    train_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y',
 'shipment_state_canceled_x',
 'shipment_state_canceled_y',
 'shipment_state_canceled',
 'shipment_state_pending_x',
 'shipment_state_pending_y',
 'shipment_state_pending',
 'shipment_state_ready_to_ship_x',
 'shipment_state_ready_to_ship_y',
 'shipment_state_ready_to_ship'
], axis=1, inplace=True)


    test_features = pd.merge(test_fold, client_features_567, on='phone_id')
    test_features[['rate_mean_new', 'rate_mean_new_x', 'rate_mean_new_y']] = test_features[
        ['rate_mean_new', 'rate_mean_new_x', 'rate_mean_new_y']].fillna(3)

    test_features = test_features.fillna(0)
    #test_features = pd.merge(test_fold, client_features_7, on='phone_id').fillna(0)
    print(len(test_fold), len(test_features), test_features.shape)
    
    test_target = deepcopy(test_features['target'])
    del test_features['target']
    del test_features['phone_id']
    del test_features['month_x']
    del test_features['month_y']
    test_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y',
 'shipment_state_canceled_x',
 'shipment_state_canceled_y',
 'shipment_state_canceled',
 'shipment_state_pending_x',
 'shipment_state_pending_y',
 'shipment_state_pending',
  'shipment_state_ready_to_ship_x',
 'shipment_state_ready_to_ship_y',
 'shipment_state_ready_to_ship'
], axis=1, inplace=True)
    
    #print(train_features.columns, train_target)
    #print(test_features.columns, test_target)

    clf.fit(train_features.values, train_target)
    print(type(clf))
    new_preds = clf.predict(test_features.values)
    score = f1_score(test_target, new_preds)
    print(score)

84812 75277 (75277, 171)
94976 83272 (83272, 171)


KeyboardInterrupt: 

## Обучаем модель на всех данных за 5, 6, 7 месяц (предсказываем 8)

In [11]:
result_clf = xgb.XGBClassifier(n_estimators=1000, max_depth=6, learning_rate=0.01, random_state=42)

test_features = pd.merge(test_data, client_features_567, on='phone_id').fillna(0)
test_features[['rate_mean_new', 'rate_mean_new_x', 'rate_mean_new_y']] = test_features[
    ['rate_mean_new', 'rate_mean_new_x', 'rate_mean_new_y']].fillna(3)
del test_features['month_y']

test_features = test_features.fillna(0)
#test_features = pd.merge(test_fold, client_features_7, on='phone_id').fillna(0)
print(len(test_fold), len(test_features), test_features.shape)

test_target = deepcopy(test_features['target'])
del test_features['target']
del test_features['phone_id']
del test_features['month_x']
del test_features['fold_num']
del test_features['order_completed_at']
test_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y',
 'shipment_state_canceled_x',
 'shipment_state_canceled_y',
 'shipment_state_canceled',
 'shipment_state_pending_x',
 'shipment_state_pending_y',
 'shipment_state_pending',
  'shipment_state_ready_to_ship_x',

 'shipment_state_ready_to_ship_y',
 'shipment_state_ready_to_ship'
], axis=1, inplace=True)

result_clf.fit(test_features, test_target)

94976 249658 (249658, 171)


KeyboardInterrupt: 

## Делаем предикт для сабмита 

In [ ]:
subm_features = pd.merge(df_test, client_features_678, how='left', 
                         left_on='Id', right_on='phone_id').fillna(0)
del subm_features['Predicted']
del subm_features['Id']
del subm_features['phone_id']
del subm_features['month_x']
del subm_features['month_y']
del subm_features['month']
#del subm_features['order_completed_at']
subm_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y',
 'shipment_state_canceled_x',
 'shipment_state_canceled_y',
 'shipment_state_canceled',
 'shipment_state_pending_x',
 'shipment_state_pending_y',
 'shipment_state_pending',
  'shipment_state_ready_to_ship_x',

 'shipment_state_ready_to_ship_y',
 'shipment_state_ready_to_ship'], axis=1, inplace=True)

result_preds = result_clf.predict(subm_features)
df_test['Predicted'] = result_preds
df_test['Predicted'] = df_test['Predicted'].astype(int)
df_test.to_csv("./HackIt_3_month_submit_super_new_feat.csv", index=False)
print(sum(result_preds))

## Блендим сабмиты разных бустингов 

In [4]:
subm_1 = pd.read_csv("./predict_proba.csv")
subm_2 = pd.read_pickle("./probas.pkl")
subm_3 = pd.read_pickle("./probas_2.pkl")

In [13]:
subm_1

,Unnamed: 0,0,1
0,0,0.457459,0.542541
1,1,0.233017,0.766983
2,2,0.773221,0.226779
3,3,0.076550,0.923450
4,4,0.705108,0.294892
...,...,...,...
214604,214604,0.166277,0.833723
214605,214605,0.613267,0.386733
214606,214606,0.313783,0.686217
214607,214607,0.936350,0.063650


In [5]:
subm_2['Predicted'] = ((subm_1['1'] + subm_2['Predicted'] +  subm_3['Predicted']) / 3)
subm_2['Predicted'] = subm_2['Predicted'].apply(lambda x: 1 if x >= 0.5 else 0)

In [22]:
subm_2

,Id,Predicted
0,19843,1
1,471287,1
2,342522,0
3,457799,1
4,233778,1
...,...,...
214604,16841,1
214605,69348,1
214606,594974,1
214607,320024,0


In [7]:
subm_2.to_csv("./blend_submit_3.csv", index=False)

In [6]:
sum(subm_2['Predicted'])

181111

In [23]:
sum(subm_2['Predicted'])

154745

In [14]:
subm_2

,Id,Predicted
0,19843,0.714797
1,471287,0.758741
2,342522,0.571473
3,457799,0.614781
4,233778,0.786192
...,...,...
214604,16841,0.686856
214605,69348,0.781341
214606,594974,0.665454
214607,320024,0.724828
